In [358]:
import json
import numpy as np
import os
import re

In [359]:
INPUT = 'output.txt'
FRAME_AMOUNT = 600
ENERGY_CONSTANT = 1000000

In [360]:
def extract_name(file_path):
    # Get the base name (e.g., 'name.txt')
    base_name = os.path.basename(file_path)
    
    # Split the name and extension and return only the name
    name, _ = os.path.splitext(base_name)
    
    return name

In [361]:
def read_file_to_list(file_path):
    lines = []
    
    try:
        # Open the file in read mode
        with open(file_path, 'r') as file:
            # Read each line and strip any trailing newlines or spaces
            lines = [line.strip() for line in file]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return lines

In [362]:
def filter_strings_by_prefix(string_list, prefix):
    # Filter the list based on whether each string starts with the given prefix
    result = [s for s in string_list if s.startswith(prefix)]
    return result

In [363]:
def extract_hardware_values(input_string):
    # Regular expression to match CPU, GPU, and EMC followed by a number
    pattern = r'CPU:\s*(\d+)\s*GPU:\s*(\d+)\s*EMC:\s*(\d+)'
    
    # Use regular expression to find the numbers
    match = re.search(pattern, input_string)
    
    if match:
        cpu_value = int(match.group(1))
        gpu_value = int(match.group(2))
        emc_value = int(match.group(3))
        return cpu_value, gpu_value, emc_value
    else:
        return None  # Return None if the pattern is not found

In [364]:
def extract_timings(input_string):
    # Regular expression to match the float numbers followed by 'ms'
    pattern = r'([\d\.]+)ms'

    # Use regular expression to find all the numbers followed by 'ms'
    matches = re.findall(pattern, input_string)
    
    # Convert the matches to float
    timings = [float(match) for match in matches]
    
    return timings

In [365]:
def extract_latency(input_string):
    # Regular expression to match a floating-point number
    pattern = r'Latency:\s*([\d\.]+)'
    
    # Use re.search to find the number after 'Latency:'
    match = re.search(pattern, input_string)
    
    if match:
        # Convert the matched value to a float
        latency = float(match.group(1))
        return latency
    else:
        return None  # Return None if no match is found

In [366]:
def extract_gpu_energy(input_string):
    # Regular expression to match a floating-point number
    pattern = r'GPU Energy Consumption:\s*([\d\.]+)'
    
    # Use re.search to find the number after 'Latency:'
    match = re.search(pattern, input_string)
    
    if match:
        # Convert the matched value to a float
        energy = float(match.group(1))
        return energy
    else:
        return None  # Return None if no match is found

In [367]:
def extract_cpu_energy(input_string):
    # Regular expression to match a floating-point number
    pattern = r'CPU Energy Consumption:\s*([\d\.]+)'
    
    # Use re.search to find the number after 'Latency:'
    match = re.search(pattern, input_string)
    
    if match:
        # Convert the matched value to a float
        energy = float(match.group(1))
        return energy
    else:
        return None  # Return None if no match is found

In [368]:
def extract_eme_energy(input_string):
    # Regular expression to match a floating-point number
    pattern = r'Memory Energy Consumption:\s*([\d\.]+)'
    
    # Use re.search to find the number after 'Latency:'
    match = re.search(pattern, input_string)
    
    if match:
        # Convert the matched value to a float
        energy = float(match.group(1))
        return energy
    else:
        return None  # Return None if no match is found

In [369]:
lines = read_file_to_list(INPUT)
y_lines = filter_strings_by_prefix(lines, "Y")
speed_lines = filter_strings_by_prefix(lines, "Speed")
latency_lines = filter_strings_by_prefix(lines, "Latency")
cpu_lines = filter_strings_by_prefix(lines, "CPU")
gpu_lines = filter_strings_by_prefix(lines, "GPU")
eme_lines = filter_strings_by_prefix(lines, "Memory")

In [370]:
iteration = len(y_lines)
cpu_freq, gpu_freq, emc_freq = extract_hardware_values(y_lines[0])

In [371]:
speed_list = []
latency_list = []
cpu_energy_list = []
gpu_energy_list = []
emc_energy_list = []
frame_rate_list = []

for i in range(iteration):
    speed_line_data = extract_timings(speed_lines[i])
    speed_line_sum = np.sum(np.array(speed_line_data))
    
    speed_list.append(speed_line_sum)
    latency_list.append(extract_latency(latency_lines[i]))
    cpu_energy_list.append(extract_cpu_energy(cpu_lines[i]))
    gpu_energy_list.append(extract_gpu_energy(gpu_lines[i]))
    emc_energy_list.append(extract_eme_energy(eme_lines[i]))
    frame_rate_list.append(FRAME_AMOUNT / extract_latency(latency_lines[i]))

cpu_power_list = np.array(cpu_energy_list) / np.array(latency_list) / ENERGY_CONSTANT
gpu_power_list = np.array(gpu_energy_list) / np.array(latency_list) / ENERGY_CONSTANT
emc_power_list = np.array(emc_energy_list) / np.array(latency_list) / ENERGY_CONSTANT
total_power_list = cpu_power_list + gpu_power_list + emc_power_list

In [372]:
speed_avg = np.average(np.array(speed_list))
speed_std = np.std(np.array(speed_list))

latency_avg = np.average(np.array(latency_list))
latency_std = np.std(np.array(latency_list))

cpu_energy_avg = np.average(np.array(cpu_energy_list))
cpu_energy_std = np.std(np.array(cpu_energy_list))

gpu_energy_avg = np.average(np.array(gpu_energy_list))
gpu_energy_std = np.std(np.array(gpu_energy_list))

emc_energy_avg = np.average(np.array(emc_energy_list))
emc_energy_std = np.std(np.array(emc_energy_list))

cpu_power_avg = np.average(np.array(cpu_power_list))
cpu_power_std = np.std(np.array(cpu_power_list))

gpu_power_avg = np.average(np.array(gpu_power_list))
gpu_power_std = np.std(np.array(gpu_power_list))

emc_power_avg = np.average(np.array(emc_power_list))
emc_power_std = np.std(np.array(emc_power_list))

frame_rate_avg = np.average(np.array(frame_rate_list))
frame_rate_std = np.std(np.array(frame_rate_list))

total_power_avg = np.average(np.array(total_power_list))
total_power_std = np.std(np.array(total_power_list))

In [373]:
result = {}
result['speed_avg'] = speed_avg
result['speed_std'] = speed_std
result['latency_avg'] = latency_avg
result['latency_std'] = latency_std
result['cpu_energy_avg'] = cpu_energy_avg
result['cpu_energy_std'] = cpu_energy_std
result['gpu_energy_avg'] = gpu_energy_avg
result['gpu_energy_std'] = gpu_energy_std
result['emc_energy_avg'] = emc_energy_avg
result['emc_energy_std'] = emc_energy_std
result['cpu_power_avg'] = cpu_power_avg
result['cpu_power_std'] = cpu_power_std
result['gpu_power_avg'] = gpu_power_avg
result['gpu_power_std'] = gpu_power_std
result['emc_power_avg'] = emc_power_avg
result['emc_power_std'] = emc_power_std
result['frame_rate_avg'] = frame_rate_avg
result['frame_rate_std'] = frame_rate_std
result['total_power_avg'] = total_power_avg
result['total_power_std'] = total_power_std

In [374]:
with open(f'{extract_name(INPUT)}.json', 'w') as file:
    json.dump(result, file, indent=4)